In [ ]:
'''ADDING PLOTTING PARAMETERS SO EXPORTED IMAGES ARE COMPATIBLE WITH ILLUSTRATOR'''

# Code from: https://jonathansoma.com/lede/data-studio/matplotlib/exporting-from-matplotlib-to-open-in-adobe-illustrator/
mlp.rcParams['pdf.fonttype'] = 42
mlp.rcParams['ps.fonttype'] = 42

# Calculation Functions Figure 8

In [50]:
def removeGenes(sampleObj, genesToRemove, condition, sample1_name, sample2_name, saveFolder):

    sampleObj = sampleObj[:, ~sampleObj.var_names.isin(genesToRemove)]
    
    # Re-Clustering Half1_A
    # PCA
    sc.tl.pca(sampleObj, svd_solver='arpack')

    # Neighbors and UMAP Computations
    sc.pp.neighbors(sampleObj, n_neighbors=50, n_pcs =50, use_rep = "Scanorama")
    sc.tl.umap(sampleObj) 

    # Re-calculating cluster numbers 
    sc.tl.leiden(sampleObj, resolution = 0.3, key_added = "leiden_0.3")

    # Calculates nuclei and gene information specific to the sample object 
    sc.pp.calculate_qc_metrics(sampleObj, percent_top=None, log1p=False, inplace=True)
    
    # Step 4: Assign the filtered AnnData object
    sampleObj.write(f'F:/SampleData/{saveFolder}/{condition}_{sample1_name}_{sample2_name}.h5ad')

# Plotting Functions Figure 8

In [25]:
def figure8(obj_sexGenes, obj_noSexGenes, comparLst, savePath):
    # Sample Info
    condition = comparLst[0]
    sample1_name = comparLst[1]
    sample2_name = comparLst[2]
    sample1_mouseSex = mouseSex(sample1_name)
    sample2_mouseSex = mouseSex(sample2_name)
    
    # Create a figure with subplots
    fig = plt.figure(figsize=(20,25), dpi=150)

    row = 6 
    col = 8

    ax1 = plt.subplot2grid((row, col), (0, 0), colspan=4, rowspan=2) # Clustered UMAP W GENES
    ax2 = plt.subplot2grid((row, col), (0, 4), colspan=4, rowspan=2) # Clustered UMAP NO GENES 
    
    ax7 = plt.subplot2grid((row, col), (3, 0), colspan=2) # DE Sample1 W GENES
    ax8 = plt.subplot2grid((row, col), (3, 2), colspan=2) # DE Sample2 W GENES

    ax9 = plt.subplot2grid((row, col), (3, 4), colspan=2) # DE Sample1 NO GENES
    ax10 = plt.subplot2grid((row, col), (3, 6), colspan=2) # DE Sample2 NO GENES
    
    if sample1_mouseSex != sample2_mouseSex:
                        
        ax3 = plt.subplot2grid((row, col), (2, 0), colspan=2) # Sample Umap Divided by Sex W GENES
        ax4 = plt.subplot2grid((row, col), (2, 2), colspan=2) # Sample Umap Divided by Sample W GENES

        ax5 = plt.subplot2grid((row, col), (2, 4), colspan=2) # Sample Umap Divided by Sex NO GENES
        ax6 = plt.subplot2grid((row, col), (2, 6), colspan=2) # Sample Umap Divided by Sample NO GENES
    
        # Sample UMAP W GENES
        sc.pl.umap(obj_sexGenes, color='leiden_0.3', legend_loc='on data', ax=ax1, show=False)
        ax1.set_title('UMAP Divided by Sex', fontsize=10)
        
        # Sample UMAP NO GENES
        sc.pl.umap(obj_noSexGenes, color='leiden_0.3', legend_loc='on data', ax=ax2, show=False)
        ax2.set_title('UMAP Divided by Sex', fontsize=10)
        
        # Sample UMAP Divided by Sex W GENES
        sc.pl.umap(obj_sexGenes, color='sex', ax=ax3, legend_loc='upper right', show=False)
        ax3.set_title('UMAP Divided by Sex', fontsize=10)
        
        # Sample UMAP Divided by Sex NO GENES
        sc.pl.umap(obj_noSexGenes, color='sex', ax=ax5, show=False)
        ax5.set_title('UMAP Divided by Sex (Sex Genes Excluded)', fontsize=10)
        
        # Sample UMAP Divided by Sample W GENES
        sc.pl.umap(obj_sexGenes, color='sample', ax=ax4, show=False)
        ax4.set_title('UMAP Divided by Sex', fontsize=10)
        
        # Sample UMAP Divided by Sex NO GENES
        sc.pl.umap(obj_noSexGenes, color='sample', ax=ax6, show=False)
        ax6.set_title('UMAP Divided by Sex (Sex Genes Excluded)', fontsize=10)
        
        # DE Graph W GENES
        obj_sexGenes.uns['log1p'] = {'base': None}
        sc.tl.rank_genes_groups(obj_sexGenes, groupby='sex', method='wilcoxon', use_raw=False)
        deGraph(obj_sexGenes, 'male', 'female', 10, ax7)
        deGraph(obj_sexGenes, 'female', 'male', 10, ax8)

        # DE Graph NO GENES
        obj_noSexGenes.uns['log1p'] = {'base': None}
        sc.tl.rank_genes_groups(obj_noSexGenes, groupby='sex', method='wilcoxon', use_raw=False)
        deGraph(obj_noSexGenes, 'male', 'female', 10, ax9)
        deGraph(obj_noSexGenes, 'female', 'male', 10, ax10)
    
    else: 
        ax3 = plt.subplot2grid((row, col), (2, 0), colspan=4) # Sample Umap Divided by Sample W GENES
        ax4 = plt.subplot2grid((row, col), (2, 4), colspan=4) # Sample Umap Divided by Sample NO GENES
    
        # Sample UMAP W GENES
        sc.pl.umap(obj_sexGenes, color='leiden_0.3', legend_loc='on data', ax=ax1, show=False)
        ax1.set_title('UMAP Divided by Sex', fontsize=10)
        
        # Sample UMAP NO GENES
        sc.pl.umap(obj_noSexGenes, color='leiden_0.3', legend_loc='on data', ax=ax2, show=False)
        ax2.set_title('UMAP Divided by Sex', fontsize=10)
        
        # Sample UMAP Divided by Sample W GENES
        sc.pl.umap(obj_sexGenes, color='sample', ax=ax3, show=False)
        ax3.set_title('UMAP Divided by Sex', fontsize=10)
        
        # Sample UMAP Divided by Sex NO GENES
        sc.pl.umap(obj_noSexGenes, color='sample', ax=ax4, show=False)
        ax4.set_title('UMAP Divided by Sex (Sex Genes Excluded)', fontsize=10)
        
        # DE Graph W GENES
        obj_sexGenes.uns['log1p'] = {'base': None}
        sc.tl.rank_genes_groups(obj_sexGenes, groupby='sample', method='wilcoxon', use_raw=False)
        deGraph(obj_sexGenes, sample1_name, sample2_name, 10, ax7)
        deGraph(obj_sexGenes, sample2_name, sample1_name, 10, ax8)

        # DE Graph NO GENES
        obj_noSexGenes.uns['log1p'] = {'base': None}
        sc.tl.rank_genes_groups(obj_noSexGenes, groupby='sex', method='wilcoxon', use_raw=False)
        deGraph(obj_noSexGenes, samle1_name, samle2_name, 10, ax9)
        deGraph(obj_noSexGenes, samle2_name, samle1_name, 10, ax10)
    
    fig.suptitle(f'{condition} - Sample {sample1_name} ({sample1_mouseSex}) vs. {sample2_name} ({sample2_mouseSex})', 
                 fontsize=20, fontweight='bold', y=0.92)
    
    # Adjust the spacing between subplots
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    
    # Saving the figure onto the computer 
    plt.savefig(os.path.join(savePath, f'{condition}_{sample1_name}_{sample2_name}.jpg'), bbox_inches = 'tight')
    

In [ ]:
def plottingFigure8(sampleComparLst, sampleFolder_sexGenes, sampleFolder_noSexGenes, savePath):
    for lst in sampleComparLst:
        condition = lst[0]
        sample1_name = lst[1]
        sample2_name = lst[2]
        
        obj_sexGenes = sc.read(f'F:/SampleData/{sampleFolder_sexGenes}/{condition}_{sample1_name}_{sample2_name}.h5ad')
        obj_noSexGenes = sc.read(f'F:/SampleData/{sampleFolder_noSexGenes}/{condition}_{sample1_name}_{sample2_name}.h5ad')
        
        figure8(obj_sexGenes, obj_noSexGenes, lst, savePath)
        